In [ ]:
from google.colab import drive

import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Misc. Functions

# Return all catagories of a data set
def setCatagories(directory):
  catagories = []
  for filename in os.listdir(directory):
    catagories.append(filename)
  return catagories

# Generator for CNN input
def gen(subset):
    while True:
        if subset == "training":
            for i in TRAINING_DATA.take(1):
                img_batch = i
            for j in TRAINING_DATA_DEPTH.take(1):
                depth_batch = j
            for k in y_test.take(1):
                labels_batch = k
        else:
            for i in TESTING_DATA.take(1):
                img_batch = i
            for j in TESTING_DATA_DEPTH.take(1):
                depth_batch = j
            for k in y_test.take(1):
                labels_batch = k
        yield ((img_batch, depth_batch), labels_batch)


In [ ]:
# Initial generation functions

# Paths to data sets
TRAINPATH =       "/content/drive/MyDrive/Dataset/Glove"
TRAINPATHDEPTH =  "/content/drive/MyDrive/Dataset/Depth"

TRAINING_DATA = None
TESTING_DATA = None

TRAINING_DATA_DEPTH = None
TESTING_DATA_DEPTH = None

# Catagories of data sets
CATAGORIES = setCatagories(TRAINPATH)

# Global veriables for data creation
IMG_SIZE = 480
BATCH_SIZE = 8

# Finds image data and applys pre-processing
def create_set(path, setType):
  dataset = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset=setType,
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)
  return dataset

# Create and process data for use in CNN
def initial_setup():
  global TRAINING_DATA, TESTING_DATA, TRAINING_DATA_DEPTH, TESTING_DATA_DEPTH

  print("Creating image training data...")
  TRAINING_DATA = create_set(TRAINPATH, "training")
  print("Image training data created!")

  print("Creating image testing data...")
  TESTING_DATA = create_set(TRAINPATH, "validation")
  print("Image testing data created!")

  print("Image data complete \n")

  print("Creating depth training data...")
  TRAINING_DATA_DEPTH = create_set(TRAINPATHDEPTH, "training")
  print("Depth training data created!")

  print("Creating depth testing data...")
  TESTING_DATA_DEPTH = create_set(TRAINPATHDEPTH, "validation")
  print("Depth testing data created!")

  print("Depth data complete \n")

initial_setup()

In [ ]:
# Tune and normalise data

AUTOTUNE = tf.data.AUTOTUNE

TRAINING_DATA = TRAINING_DATA.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
TESTING_DATA = TESTING_DATA.cache().prefetch(buffer_size=AUTOTUNE)

TRAINING_DATA_DEPTH = TRAINING_DATA_DEPTH.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
TESTING_DATA_DEPTH = TESTING_DATA_DEPTH.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = TRAINING_DATA.map(lambda x, y: (normalization_layer(x), y))
normalized_ds_depth = TRAINING_DATA_DEPTH.map(lambda x, y: (normalization_layer(x), y))

y_train = tf.data.Dataset.from_tensor_slices(tf.random.uniform((262,1))).batch(BATCH_SIZE)
y_test = tf.data.Dataset.from_tensor_slices(tf.random.uniform((65,1))).batch(BATCH_SIZE)

In [ ]:
# Apply data to generator
gen_train = gen("training")
gen_valid = gen("validation")

In [ ]:
# shapes as you said in comments
img_data_shape = (480, 480, 3)
dep_data_shape = (480, 480, 3)
num_classes = 3

# define two inputs layers
img_input = tf.keras.layers.Input(shape=img_data_shape, name="image")
depth_input = tf.keras.layers.Input(shape=dep_data_shape, name="depth")

# Layers for image data 
x1 = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(img_input)
x1 = tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', name="conv1_img")(x1)
x1 = tf.keras.layers.MaxPooling2D(name="maxp1_img")(x1)
x1 = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', name="conv2_img")(x1)
x1 = tf.keras.layers.MaxPooling2D(name="maxp2_img")(x1)
x1 = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', name="conv3_img")(x1)
x1 = tf.keras.layers.MaxPooling2D(name="maxp3_img")(x1)
x1 = tf.keras.layers.Flatten(name="flatten_img")(x1)


# Layers for depth data
x2 = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(depth_input)
x2 = tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', name="conv1_depth")(x2)
x2 = tf.keras.layers.MaxPooling2D(name="maxp1_depthg")(x2)
x2 = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', name="conv2_depth")(x2)
x2 = tf.keras.layers.MaxPooling2D(name="maxp2_depth")(x2)
x2 = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', name="conv3_depth")(x2)
x2 = tf.keras.layers.MaxPooling2D(name="maxp3_depth")(x2)
x2 = tf.keras.layers.Flatten(name="flatten_depth")(x2)

# Merge
x = tf.keras.layers.concatenate([x1,x2], name="concat_depth_img")
x = tf.keras.layers.Dense(128, activation='relu', name="dense1_both")(x)
output = tf.keras.layers.Dense(num_classes, name="classify")(x)

# Model for two inputs and an output
model = tf.keras.models.Model(inputs=[img_input, depth_input], outputs=output)
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Train Model
histroy = model.fit(gen_train, epochs=20, steps_per_epoch=3, validation_data=gen_valid, validation_steps=1)

In [ ]:
#Save Model
model.save('/content')